# Time-varying weathering in Texture Space

Implementation of the paper https://www.cs.tau.ac.il/~dcor/articles/2016/TW.pdf.

In [ ]:
# !pip3 install memory_profiler opencv-python numpy matplotlib scikit-image

# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import skimage.color
import skimage.filters
import skimage.io
import skimage.feature
import skimage.measure
import skimage.color

from textureTransfer import Construct as transfer_texture
%matplotlib inline
%load_ext memory_profiler

## Utility functions

In [ ]:
# Scaling or clipping
scale = lambda x, top=255: (top * (x - np.min(x))) / (np.max(x) - np.min(x))
inrange = lambda x: np.where(x > 255, 255, np.where(x < 0, 0, x))
invert = lambda x: np.max(x) - x
print_range = lambda x: print(np.min(x), np.max(x))

In [ ]:
prewitt = [np.array(
               [[-1, 0, 1],
                [-1, 0, 1],
                [-1, 0, 1]]), 
           np.array(
               [[ 1,  1,  1],
                [ 0,  0,  0],
                [-1, -1, -1]])]

def apply_filter(im, filt, zero_padding=False):
    '''
    Filter Convolves filter over the input image
    
    im: Input Image
    filt: Filter
    zero_padding: True will result in zero padding, 
                  False will result in padding by borders
    '''
    
    wdth = filt.shape[0]//2
    
    filt_img = np.zeros(im.shape)
    
    if zero_padding:
        im = np.vstack( (np.zeros( (wdth, im.shape[1]) ), im, np.zeros( (wdth, im.shape[1]) ) ) )
        im = np.hstack( (np.zeros( (im.shape[0], wdth) ), im, np.zeros( (im.shape[0], wdth) ) ) )
    else:
        im = np.vstack(([im[0]]*wdth, im, [im[-1]]*wdth))
        im = np.hstack((np.hstack([im[:,0].reshape(-1,1)]*wdth), im, np.hstack([im[:,-1].reshape(-1,1)]*wdth)))

    for i in range(wdth, im.shape[0]-wdth):
        for j in range(wdth, im.shape[1]-wdth):
            filt_img[i-wdth][j-wdth] = np.sum(im[i-wdth:i+wdth+1, j-wdth:j+wdth+1] * filt)
    
    return np.rint(filt_img)

In [ ]:
def median_filter(im, k):
    '''
    Function to run median filter on input image
    
    im: Input image
    k: Kernel size
    '''
        
    filt_img = np.zeros(im.shape)

    wdth = k//2

    im = np.vstack(([im[0]]*wdth, im, [im[-1]]*(k-wdth)))
    im = np.hstack((np.hstack([im[:,0].reshape(-1,1)]*wdth), im, np.hstack([im[:,-1].reshape(-1,1)]*(k-wdth))))

    for j in range(wdth, im.shape[1]-wdth):
        for i in range(wdth, im.shape[0]-wdth):
            x = np.median(im[i-wdth:i+k-wdth+1, j-wdth:j+k-wdth+1])
            filt_img[i-wdth][j-wdth] = x
    
    return filt_img

In [ ]:
def decouple(img, type=0):
    '''
    Returns the luminance and gradient of input image
    
    img: Input Image
    type: Flag which determines the output: 0 for luminance and gradient
                                            1 for only gradient
                                            2 for only luminance
    '''
    
    img = img.astype('float64')
    
    with np.errstate(divide='ignore', invalid='ignore'):
        intensity_layer = np.nan_to_num(np.true_divide(((img[:, :, 0]**2) + (img[:, :, 1]**2) + (img[:, :, 2]**2)) , (img[:,:,0]+img[:, :, 1]+img[:, :, 2])))
        intensity_layer = np.rint(intensity_layer).astype('uint8')
        
        if type < 2:
            dx = apply_filter(intensity_layer.astype('float64'), prewitt[0])
            dy = apply_filter(intensity_layer.astype('float64'), prewitt[1])
        
            gradient = np.nan_to_num(np.arctan(dy/dx))
        else:
            return intensity_layer
    
    if type == 1:
        return gradient
    return intensity_layer,gradient

## Distance

In [ ]:
def distance(sI, sG, tI, tG):
    '''
    Computes the distance map between source patch and all possible patches
    
    sI: Source Patch luminance
    sG: Source Patch gradient
    tI: Target Image luminance
    tG: Target Image gradient
    '''

    M, N = tI.shape
    
    Y, X = sI.shape
    wdth_y, wdth_x = Y//2, X//2

    tI = np.vstack(([tI[0]]*wdth_y, tI, [tI[-1]]*(Y-wdth_y)))
    tI = np.hstack((np.hstack([tI[:,0].reshape(-1,1)]*wdth_x), tI, np.hstack([tI[:,-1].reshape(-1,1)]*(X-wdth_x))))

    tG = np.vstack(([tG[0]]*wdth_y, tG, [tG[-1]]*(Y-wdth_y)))
    tG = np.hstack((np.hstack([tG[:,0].reshape(-1,1)]*wdth_x), tG, np.hstack([tG[:,-1].reshape(-1,1)]*(X-wdth_x))))
   
    g_img = np.zeros((M, N))
    l_img = np.zeros((M, N))
   
    for i in range(wdth_y, wdth_y+M):
        for j in range(wdth_x, wdth_x+N):
            try:
                g_img[i-wdth_y ,j-wdth_x] = np.average(np.abs(sG - tG[i-wdth_y:i+Y-wdth_y, j-wdth_x:j+X-wdth_x]))
                l_img[i-wdth_y ,j-wdth_x] = np.average(np.abs(sI - tI[i-wdth_y:i+Y-wdth_y, j-wdth_x:j+X-wdth_x]))
            
            except Exception:
                print("Source shape: ", sG.shape, "  i, j:", i, j, "   Patch shape:",  tG[i-wdth_y:i+Y-wdth_y, j-wdth_x:j+X-wdth_x].shape)
                raise Exception
            
    return (g_img / np.max(g_img)) + (l_img / np.max(l_img))

In [ ]:
def inbuilt_distance(sI, sG, tI, tG, scalefactor=True):
    '''
    Computes the distance map between source patch and all possible patches
    (Uses Corelation-Coeficient template matching from opencv)
    
    sI: Source Patch luminance
    sG: Source Patch gradient
    tI: Target Image luminance
    tG: Target Image gradient
    '''

    M, N = tI.shape
    
    Y, X = sI.shape
    wdth_y, wdth_x = Y//2, X//2
    try:
        if wdth_y:
            tI = np.vstack(([tI[0]]*wdth_y, tI))
            tG = np.vstack(([tG[0]]*wdth_y, tG))
        if Y-wdth_y-1:
            tI = np.vstack((tI, [tI[-1]]*(Y-wdth_y-1)))
            tG = np.vstack((tG, [tG[-1]]*(Y-wdth_y-1)))
        
        if wdth_x:
            tI = np.hstack((np.hstack([ tI[:,0].reshape(-1,1)]*wdth_x), tI))
            tG = np.hstack((np.hstack([tG[:,0].reshape(-1,1)]*wdth_x), tG))
        if X-wdth_x-1:
            tI = np.hstack((tI, np.hstack([tI[:,-1].reshape(-1,1)]*(X-wdth_x-1))))
            tG = np.hstack((tG, np.hstack([tG[:,-1].reshape(-1,1)]*(X-wdth_x-1))))
            
    except Exception:
        print("\n", wdth_x, wdth_y, Y-wdth_y-1, X-wdth_x-1)
        raise Exception

    lum = cv2.matchTemplate(tI,sI, cv2.TM_CCOEFF)
    grd = cv2.matchTemplate(np.rint(scale(tG)).astype('uint8'), np.rint(scale(sG)).astype('uint8'), cv2.TM_CCOEFF)

    if scalefactor:
        return scale(invert(lum+grd), 1)
    
    return invert(lum+grd)

## Age Map

In [ ]:
def age(source_patch, target, K, num_bins=1000):
    '''
    Calculates the age of a source patch
    
    source_patch: Tuple with source patch luminance and gradient layers
    target: Tuple with target luminance and gradient layers
    K: Percentage of pixels to consider for thresholding
    num_bins: Number of levels to threshhold into
    '''
    
    sI, sG = source_patch
    tI, tG = target
    dist_mat = inbuilt_distance(sI, sG, tI, tG, False)

    # # Method 1
    # discrete = np.rint(scale(dist_mat)).astype('uint8')
    # size = sI.shape[0]//2
    # size = max(1, size + (1-(size%2)))
    # discrete = cv2.medianBlur(discrete, size)

    # K_distances = np.sort(np.unique(discrete).flatten())[:K]
    # co_ords = []
    # for i in K_distances:
    #     x, y = np.where(discrete == i)
    #     for j in zip(x, y):
    #         co_ords.append(j)

    # K_neighbours = [dist_mat[x, y] for (x, y) in co_ords]
    # K_neighbours = [dist_mat[x, y] for (x, y) in co_ords]

    # return np.average(K_neighbours)
    
    # Method 2
    bins = np.digitize(dist_mat, np.linspace(np.min(dist_mat), np.max(dist_mat), num_bins))
    bins_mapped = np.linspace(np.min(dist_mat), np.max(dist_mat), num_bins)[bins-1]
    vals, counts = np.unique(bins_mapped, return_counts=True)
    cumsum = np.cumsum(counts)
    cumsum = cumsum/cumsum[-1]
    x, = np.argwhere(cumsum > K)[0]

    
    K_distances = vals[:x] * counts[:x]
    K_distances = np.sum(K_distances)/np.sum(counts[:x])

    return K_distances

In [ ]:
def generate_age_map(I, G, N, K):
    '''
    Generates the age map of input image
    
    I: Input image luminance
    G: Input image gradient
    N: Patch Size
    K: Percentage of pixels to consider for thresholding
    '''

    age_map = np.zeros(I.shape, dtype=('float64'))

    for i in range(0, I.shape[0], N):
        for j in range(0, I.shape[1], N):
            x1, x2 = i, min(I.shape[0], i+N)
            y1, y2 = j, min(I.shape[1], j+N)
            age_map[x1:x2, y1:y2] = age( (I[x1:x2, y1:y2], G[x1:x2, y1:y2]) , (I, G), K)

    return age_map

## Color Map

In [ ]:
import matplotlib.colors as mcolors


def make_colormap(seq):
    '''
    Return a LinearSegmentedColormap
    
    seq: a sequence of floats and RGB-tuples. The floats should be increasing
         and in the interval (0,1).
    '''
    
    seq = [(None,) * 3, 0.0] + list(seq) + [1.0, (None,) * 3]
    cdict = {'red': [], 'green': [], 'blue': []}
    for i, item in enumerate(seq):
        if isinstance(item, float):
            r1, g1, b1 = seq[i - 1]
            r2, g2, b2 = seq[i + 1]
            cdict['red'].append([item, r1, r2])
            cdict['green'].append([item, g1, g2])
            cdict['blue'].append([item, b1, b2])
    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

c = mcolors.ColorConverter().to_rgb

heatmap = make_colormap([c('blue'), c('aqua'), 0.33, c('aqua'), c('yellow'), 0.66, c('yellow'), c('red')])

## Intact texture generation : Tile Detection

In [ ]:
def global_minimas(dist_map):
    '''
    Computes global minimas in distance map
    
    dist_map: Distance map
    '''
    
    a,b=dist_map.shape
    minima=np.zeros((a,b),dtype='uint8')
    
    thresh=np.unique(dist_map)[np.rint(0.005*(a-1)*(b-1)).astype('uint8')]
    minima=dist_map<thresh     
                      
    return minima.astype('uint8')     

def local_minimas(dist_map,N):
    '''
    Computes local minimas in distance map
    
    dist_map: Distance map
    '''
    
    a,b=dist_map.shape
    minima=np.zeros((a,b),dtype='uint8')
    for i in range(0,a,N):
        for j in range(0,b,N):
            X=min(i+N,a)
            Y=min(j+N,b)
            patch=dist_map[i:X,j:Y]
            thresh=np.unique(patch)[np.rint(0.005*(X-i)*(Y-j)).astype('uint8')]
            minima[i:X,j:Y]=patch<thresh     
                      
    return minima.astype('uint8')

In [ ]:
def make_tile(luminance, gradient,labels):
    '''
    Finds potential tile centers in the input image
    
    luminance: Luminance of Input image
    gradient: Gradient of Input image
    labels: label map for connected components for thresholded distance map
    '''
    
    a,b = luminance.shape
    N1 = a//4
    N2 = b//4
    
    check=np.unique(labels)[1:]
    tiles={}
    global_min={}
    
    for i in check:
        temp = np.where(labels==i)
        l = len(temp[0])
        x = temp[0][l//2]
        y = temp[1][l//2]
        grad_tile = gradient[ max(0,x-N1//2):min(x+N1//2,a-1) , max(0,y-N2//2):min(b-1,y+N2//2) ]
        lum_tile = luminance[ max(0,x-N1//2):min(x+N1//2,a-1) , max(0,y-N2//2):min(b-1,y+N2//2) ]

        tiles[i] = inbuilt_distance(lum_tile, grad_tile, luminance, gradient)
        global_min[i] = global_minimas(tiles[i])
        
    return global_min

In [ ]:
def sum_tiles(pot_tiles):
    '''
    Takes aggregation of potential tile centers detected by make_tile
    
    pot_tiles: list of images with potential centres found using various patches
    '''
    
    a,b=pot_tiles[1].shape
    sum_tiles=np.zeros((a,b),dtype='uint8')

    for i in pot_tiles:
        sum_tiles+=pot_tiles[i]
        
    return sum_tiles    

In [ ]:
def detect_peaks_local(sum_tiles):
    '''
    Detects the peaks in the aggregation of centers to find most potent centers
    
    sum_tiles: aggregation of potential centers created by sum_tiles
    '''
    
    a,b = sum_tiles.shape
    N1 = a//4
    N2 = b//4
    maxima = np.zeros((a,b),dtype='uint8')
    
    for i in range(0,a,N1):
        for j in range(0,b,N2):
            X = min(i+N1,a)
            Y = min(j+N2,b)
            patch = sum_tiles[i:X,j:Y]
            thresh = np.max(patch)

            if thresh > 0:
                maxima[i:X,j:Y] = patch == thresh
                maxima[maxima == 1] = 255
            else:
                maxima[i:X,j:Y] = patch      

    return N1,N2,maxima.astype('uint8')                  

In [ ]:
## Finding the Offset Vectors

def grid_calc(final_tile_centers, N1, N2):
    '''
    Defines the grid structure in the image using most potent centers
    
    final_tile_centers: Peak values of centers found using detect_peaks_local
    '''
    
    Tile_centers = []
    x,y = np.nonzero(final_tile_centers)[0],np.nonzero(final_tile_centers)[1]

    for c in range(x.shape[0]):
        Tile_centers.append([x[c] , y[c]])
    Tile_centers = np.array(Tile_centers)
    Offset_Vectors = []
    
    for i in range(Tile_centers.shape[0]):
        for j in range(i+1,Tile_centers.shape[0]):
                Offset_Vectors.append(Tile_centers[i]- Tile_centers[j])
                
    # Offset_Vectors = np.array(Offset_Vectors)
    u,c = np.unique(Offset_Vectors,axis = 0, return_counts=True)
    sorted_arr = u[c.argsort()]
        
    v=abs(sorted_arr)
    
    v=v[ np.where( np.any([np.all(v >= [N1/2, N2/2], axis=1), np.logical_and(v[:, 0] == 0, v[:, 1] >= N2/2), np.logical_and(v[:, 0] >= N1/2, v[:, 1] == 0)], axis=0 ) ) ][-15:]
    counts = c[c.argsort()]
    counts = counts[np.where( np.any([np.all(v >= [N1/2, N2/2], axis=1), np.logical_and(v[:, 0] == 0, v[:, 1] >= N2/2), np.logical_and(v[:, 0] >= N1/2, v[:, 1] == 0)], axis=0 ) ) ][-15:]
  
    try:
        x, _ = v[np.argwhere(v[:, 1] == 0)][-1][0]
    except:
        x, _ = v[np.argwhere(v[:, 0] > 0)][-1][0]
    try:
        _, y = v[np.argwhere(v[:, 0] == 0)][-1][0]
    except:
        _, y = v[np.argwhere(v[:, 1] > 0)][-1][0]

    return x, y

## Intact texture generation : Median Image

In [ ]:
def median_image(input_texture, x1, y1): 
    '''
    Creates the median image of input texture
    
    input_texture: Input image
    x1: height of grid
    y1: width of grid
    '''
    
    tiles = []

    for i in range(x1):
        for j in range(y1):
            xi = i*x1
            xf = (i+1) * x1
            yi = j*y1
            yf = (j+1) * y1

            if (xf < input_texture.shape[0]) and (yf < input_texture.shape[1]):
                tiles.append(input_texture[xi:xf, yi:yf])

    medianTile = np.rint(np.median(tiles, axis=(0)))
    medianImage = np.copy(input_texture)

    for i in range(x1):
        xi = i*x1
        xf = (i+1) * x1
        if xi >= medianImage.shape[0]:
            break
        for j in range(y1):
            yi = j*y1
            yf = (j+1) * y1

            if yi >= medianImage.shape[1]:
                break

            if (xf < medianImage.shape[0]) and (yf < medianImage.shape[1]):
                medianImage[xi:xf, yi:yf] = medianTile
            else:
                xf2 = min(xf, medianImage.shape[0])
                yf2 = min(yf, medianImage.shape[1])
                medianImage[xi:xf2, yi:yf2] = medianTile[:(xf2-xi), :(yf2-yi)]
                
    return medianImage            

## Intact texture generation : Template Generation

In [ ]:
def intact_template(img_name):
    '''
    Generates the intact template of input image
    
    img_name: A numpy array (Input Image) or string (Filename of Input Image)
    '''
    
    path='../images/'
    
    img=cv2.imread(path+img_name)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    a,b,c=img.shape
    N = a//4

    details, gradient = decouple(img)
    source_patch_one = img[a//4:a//4+N,b//4:b//4+N, :]
    source_patch_l = details[a//4:a//4+N,b//4:b//4+N]
    source_patch_g = gradient[a//4:a//4+N,b//4:b//4+N]

    dist_map = inbuilt_distance(source_patch_l, source_patch_g, details, gradient)
    
    g=global_minimas(dist_map)
    g_bin= cv2.threshold(g, 0, 1, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    ret, labels = cv2.connectedComponents(g_bin)
    
    pot_tiles=make_tile(details,gradient,labels)
    sum_=sum_tiles(pot_tiles)   
    
    N1,N2,final_tile_centers = detect_peaks_local(sum_)
    x,y=grid_calc(final_tile_centers, N1, N2)
    print(min(x, y))
    med_img=median_image(img,x,y)
    
    plt.figure(figsize=(20,20))
    plt.subplot(121)
    plt.imshow(img)
    plt.title("Input Image")
    plt.subplot(122)
    plt.imshow(med_img)
    plt.title("Median Image")
    plt.show()

    return med_img

## Intact texture generation : Texture Transfer (Quilting)

In [ ]:
import importlib
import textureTransfer
importlib.reload(textureTransfer)
transfer_texture = textureTransfer.Construct

In [ ]:
def transfer(input_texture, intact_template, blocksize, N=5, old=None,stochastic_mask = None):
    '''
    Quilting Algorithm for Texture transfer
    
    input_texture: Source texture
    intact_template: Target Image
    blocksize: Block Size for texture transfer
    N: iterations for Texture Transfer process
    old: Previous Iteration image for comparison
    stochastic_mask: Mask for removing most weathered patches from texture
    '''
    if type(input_texture) == type('str'):
        img0 = cv2.imread('../images/'+input_texture)
        img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    else:
        img0 = input_texture
    
    for i in range(5):
            
        blocksize_new = max(12, blocksize//3)
            
        alpha_i = (0.8 * (i/(N-1))) + 0.1
        if i==0:
            t = transfer_texture(img0, intact_template, [blocksize_new, blocksize_new], blocksize_new//6, alpha=alpha_i, tolerance=0.001, compareImage=None,stochastic_mask = None)    
        else:    
            t = transfer_texture(img0, intact_template, [blocksize_new, blocksize_new], blocksize_new//6, alpha=alpha_i, tolerance=0.001, finalImage=t, compareImage=None,stochastic_mask = None)   
    
    return t.astype('uint8')

## Intact texture generation : Helper Functions

In [ ]:
def intact_texture(img, size=None, ret_bsize=False, ret_medImg=False):
    '''
    Generates the intact Texture from input image
    
    img:  A numpy array (Input Image) or string (Filename of Input Image)
    size: If given, images will be plotted with figsize=size
    ret_bsize: Flag to return blocksize in intact texture
    ret_medImg: Flag to return median Image
    '''
    
    if type(img) == type('str'):
        img=cv2.imread('../images/'+img)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
    a,b,c=img.shape
    N = a//4

    details, gradient = decouple(img)
    source_patch_one = img[a//4:a//4+N,b//4:b//4+N, :]
    source_patch_l = details[a//4:a//4+N,b//4:b//4+N]
    source_patch_g = gradient[a//4:a//4+N,b//4:b//4+N]

    dist_map = inbuilt_distance(source_patch_l, source_patch_g, details, gradient)
    
    g=global_minimas(dist_map)
    g_bin= cv2.threshold(g, 0, 1, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    ret, labels = cv2.connectedComponents(g_bin)
    
    pot_tiles=make_tile(details,gradient,labels)
    sum_=sum_tiles(pot_tiles)   
    
    N1,N2,final_tile_centers = detect_peaks_local(sum_)
    x,y=grid_calc(final_tile_centers, N1, N2)
    med_img=median_image(img,x,y)
    
    intact = transfer(img, med_img, (min(x, y)))
    
    if size is not None:
        plt.figure(figsize=size)
        plt.subplot(131).imshow(img)
        plt.title("Input Texture")
        plt.subplot(132).imshow(med_img)
        plt.title("Intact Template")
        plt.subplot(133).imshow(intact)
        plt.title("Intact Texture")
        plt.show()
    
    if ret_medImg and ret_bsize:
        return intact, min(x, y), med_img
    elif ret_medImg:
        return intact, med_img
    elif ret_bsize:
        return intact, min(x, y)
    else:
        return intact

In [ ]:
def negate(img):
    return img * (-1) + 255

def stoc_texture(img_name, size=None, ret_bsize=False):
    '''
    Compute stochastic intact texture
    
    img_name:  A numpy array (Input Image) or string (Filename of Input Image)
    size: If given, images will be plotted with figsize=size
    ret_bsize: Flag to return blocksize
    '''
    
    if type(img_name) == type('str'):
        img=cv2.imread('../images/'+img_name)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        img = img_name
        
    a,b,c=img.shape
    N = a//4

    details, gradient = decouple(img)
    source_patch_one = img[a//4:a//4+N,b//4:b//4+N, :]
    source_patch_l = details[a//4:a//4+N,b//4:b//4+N]
    source_patch_g = gradient[a//4:a//4+N,b//4:b//4+N]

    dist_map = inbuilt_distance(source_patch_l, source_patch_g, details, gradient)
    g=global_minimas(dist_map)
    g_bin= cv2.threshold(g, 0, 1, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    ret, labels = cv2.connectedComponents(g_bin)
    
    pot_tiles=make_tile(details,gradient,labels)
    sum_=sum_tiles(pot_tiles)   
    
    N1,N2,final_tile_centers = detect_peaks_local(sum_)
    x,y=grid_calc(final_tile_centers, N1, N2)
    med_img=median_image(img,x,y)

    age_map = generate_age_map(details, gradient, 20, 100/np.product(details.shape))
    h,w =age_map.shape
    uniq =  np.unique(age_map)
    thresh = uniq[np.rint(0.8*h*w).astype('uint8')]
    mask = np.zeros(dist_map.shape)
    mask[age_map > thresh ] = 255
    mask = negate(mask)
    plt.imshow(mask,cmap='gray')
    plt.show()

    intact = transfer(img, med_img, (min(x, y)) ,stochastic_mask = mask)
    
    if size is not None:
        plt.figure(figsize=size)
        plt.subplot(131).imshow(img)
        plt.title("Input Texture")
        plt.subplot(132).imshow(med_img)
        plt.title("Intact Template")
        plt.subplot(133).imshow(intact)
        plt.title("Intact Texture")
        plt.show()
    
    if ret_bsize:
        return intact, min(x, y)
    else:
        return intact

## Deweathering

In [ ]:
def deweather(input_image, t, details=None, gradient=None, int_tex=None, bsize=None, age_map=None, t_old=None, size=None):
    '''
    Generate deweathered image at time t
    
    input_image:  A numpy array (Input Image) or string (Filename of Input Image)
    t: time (0<t<1)
    details: luminance of input image
    gradient: gradient of input image
    int_tex: intact texture of input image
    bsize: Block size in intact image
    age_map: Age map of input image
    t_old: Deweathered image from previous iteration
    size: If given, images will be plotted with figsize=size
    '''
    
    if type(input_image) == type('str'):
        input_texture = cv2.imread('../images/'+input_image)
        input_texture = cv2.cvtColor(input_texture, cv2.COLOR_BGR2RGB)
    else:
        input_texture = input_image
    
    if details is None or gradient is None:
        details, gradient = decouple(input_texture)
    
    if age_map is None:
        print("\r\t\t\t\t\t\t\t\t\rGenerating Age Map...\t\t")
        age_map = generate_age_map(details, gradient, 20, 100/np.product(details.shape))
        age_map = cv2.GaussianBlur(age_map,(41, 41),0)
        age_map = scale(age_map, 1)
        print("\r\t\t\t\t\t\t\t\t\rDone!\t\t\t\t")
    age_map_t =  np.where(age_map < (1-t), 0, age_map - (1-t))
    
    if int_tex is None or bsize is None:
        print("\r\t\t\t\t\t\t\t\t\rGenerating Intact_texture...\t\t")
        int_tex, bsize = intact_texture(input_texture, ret_bsize=True)
        print("\r\t\t\t\t\t\t\t\t\rDone!\t\t\t\t")
    
    template_t = (input_texture * np.dstack((age_map_t, age_map_t, age_map_t))) + (int_tex * np.dstack((1-age_map_t, 1-age_map_t, 1-age_map_t)))
    
    print("\r\t\t\t\t\t\t\t\t\rGenerating Deweathered Image...\t\t")
    deweathered = transfer(input_texture, template_t, bsize, old=t_old)
    print("\r\t\t\t\t\t\t\t\t\rDone!\t\t\t\t")
    
    print("\r\t\t\t\t\t\t\t\t\t\t\t\t\t\r")
    
    if size is not None:
        plt.figure(figsize=size)
        plt.subplot(131).imshow(input_texture)
        plt.subplot(132).imshow(age_map_t, cmap=heatmap)
        plt.subplot(133).imshow(deweathered)
        plt.show()
    
    return deweathered

In [ ]:
def generate_deweathering_gif(input_image_name, nframes=30):
    
    import matplotlib.animation as anim
    
    if type(input_image_name) != type('str'):
        print("Please provide the File Name")
        return
    
    img = cv2.imread('../images/'+input_image_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    details, gradient = decouple(img)
    
    age_map = generate_age_map(details, gradient, 20, 100/np.product(details.shape))
    age_map = cv2.GaussianBlur(age_map,(41, 41),0)
    age_map = scale(age_map, 1)
    
    int_tex, bsize = intact_texture(img, ret_bsize=True)
    
    timespace = np.linspace(0, 1, nframes)
    images = np.array([None for i in timespace])
    
    images[0] = deweather(img, timespace[0], details=details, gradient=gradient, int_tex=int_tex, bsize=bsize, age_map=age_map, t_old=None, size=None)
    for i in range(1, len(timespace)):
        print("Frame ", i+1, ".....")
        images[i] = deweather(img, timespace[i], details=details, gradient=gradient, int_tex=int_tex, bsize=bsize, age_map=age_map, t_old=images[i-1], size=None)

    return images

def gen_gif(input_image_name, images=None, nFrames=30):
    if images is None:
        images = generate_deweathering_gif(input_image_name, nframes)

    fig = plt.figure()

    frames = []
    for i in images:
        frames.append([plt.imshow(i, animated=True)])
    nframes = len(images)
    gif = anim.ArtistAnimation(fig, frames, interval=500, blit=True, repeat_delay=100)

    gif.save('../images/'+'deweathering_gif_'+input_image_name+'.gif')
    print("GIF generated")
    fig.clear()
    
    return

# images_input = generate_deweathering_gif('input.png')
# gen_gif('input.png', images_input)

## Weathering

In [ ]:
def Qxy(centres, N, i=0):
    '''
    Calculates coordinates of patches around center coordinates
    
    centers: list with center co-ordinates
    N: Size of patch
    i: list index to select
    '''
    
    return centres[0][i], centres[0][i]+N, centres[1][i], centres[1][i]+N

def weathering(input_image, age_map=None, intact_texture_img=None, bsize=None, use_median_age=False, do_texture_transfer=True, size=(20,5), show_debug_output=False):
    '''
    Generate weathered image
    
    input_image:  A numpy array (Input Image/previous weathered image) or string (Filename of Input Image))
    intact_texture_img: intact texture of input image
    bsize: Block size in intact image
    age_map: Age map of input image
    size: If given, images will be plotted with figsize=size
    use_median_age: Whether to use median or average for age selection
    do_texture_transfer: Flag for texture transfer
    show_debug_output: Flag for debugging outputs
    '''
    
    if type(input_image) == type('str'):
        input_texture = cv2.imread('../images/' + input_image)
        input_texture = cv2.cvtColor(input_texture, cv2.COLOR_BGR2RGB)
    else:
        input_texture = input_image.copy()

    inpDetails, inpGrad = decouple(input_texture)
    
    if age_map is None:
        print("\r\t\t\t\t\t\t\t\t\rGenerating Age Map...\t\t")
        age_map = generate_age_map(inpDetails, inpGrad, 20, 100/np.product(inpDetails.shape))
        age_map = cv2.GaussianBlur(age_map,(41, 41), 0)
        age_map = scale(age_map, 1)
        print("\r\t\t\t\t\t\t\t\t\rDone!\t\t\t\t")

    if intact_texture_img is None or bsize is None:
        print("\r\t\t\t\t\t\t\t\t\rGenerating Intact_texture...\t\t")
        intact_texture_img, bsize = intact_texture(input_texture, ret_bsize=True)
        print("\r\t\t\t\t\t\t\t\t\rDone!\t\t\t\t")

    intactDetails, intactGrad = decouple(intact_texture_img)

    # Generating Delta Map
    delta_map = scale((inpDetails-intactDetails) + (inpGrad-intactGrad), 1)
    
    if show_debug_output:
        print("Delta Map Made")

    a, b, _ = input_texture.shape
    N = a//6
    x1, y1 = np.random.randint(0,a-N), np.random.randint(0,b-N)
    x2, y2 = x1 + N, y1 + N

    source_patch = input_texture[x1:x2, y1:y2, :]
    if use_median_age:
        m = np.median(delta_map[x1:x2, y1:y2])
    else:
        m = np.average(delta_map[x1:x2, y1:y2])
        
    if show_debug_output:
        print('m:', m)

    m2 = max(1, m + 0.1*m)
    centres = np.where(np.logical_and(age_map > m, age_map <= m2))
    
    # Chosing random centers
    i = np.random.randint(0, len(centres[0])-1)
    Q_x1, Q_x2, Q_y1, Q_y2 = Qxy(centres, N, i)
    
    # Calculating Edge Map
    sobelx = cv2.Sobel(intactDetails,cv2.CV_64F,1,0,ksize=3)  # x
    sobely = cv2.Sobel(intactDetails,cv2.CV_64F,0,1,ksize=3) 
    edge_map2=np.hypot(sobelx,sobely)
    edge_map=cv2.threshold(scale(edge_map2).astype('uint8'), 0, 1, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    if show_debug_output:
        print("Edge Map Made")

    # Finding Neighborhood
    neighborhood = [(x1, x1+N, y1, y1+N) for x1 in range(Q_x1-N, Q_x2+N) for y1 in range(Q_y1-N, Q_y2+N)]
    source_patch_edge = edge_map[x1:x2, y1:y2]

    min_diff = 100000
    target_patch = (0,0,0,0)

    for n in neighborhood:
        if n[1] >= a or n[3] >= b or n[0] < 0 or n[2] < 0:
            continue
        tmp_patch_edge = edge_map[n[0]:n[1], n[2]:n[3]]
        if np.sum((tmp_patch_edge - source_patch_edge)**2) < min_diff:
            target_patch = n
            min_diff = np.sum((tmp_patch_edge - source_patch_edge)**2)

    if show_debug_output:
        print("Neighbourhood calc done")
        
    out = np.copy(input_texture)
    dm = delta_map[target_patch[0]:target_patch[1], target_patch[2]:target_patch[3]]
    dm = np.dstack((dm, dm, dm))
    textureSynth = input_texture[x1:x2, y1:y2] * (1-dm) + input_texture[target_patch[0]:target_patch[1], target_patch[2]:target_patch[3]] * dm
    out[x1:x2, y1:y2] = textureSynth
    
    if do_texture_transfer:
        out = transfer(input_texture, out, N)
    
    print("\r\t\t\t\t\t\t\t\t\t\t\t\t\t\r")  
    
    if size is not None:
        plt.figure(figsize = size)
        plt.subplot(121)
        plt.imshow(input_texture)
        plt.subplot(122)
        plt.imshow(out)
        plt.show()
        
    return out

# It's SHOWTIME!

In [ ]:
def run(img_paths, write_output=False):
    '''
    Produce outputs for images in imgPaths
    
    img_paths: list of all image paths
    write_output: Whether to write image outputs to disk
    '''
    
    for img in img_paths:
        print("Processing", img.split('/')[-1])
        input_image = cv2.imread(img)
        input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

        details, gradient = decouple(input_image)

        age_map = generate_age_map(details, gradient, 20, 100/np.product(details.shape))
        age_map = cv2.GaussianBlur(age_map,(41, 41),0)
        age_map = scale(age_map, 1)

        input_intact, block_size = intact_texture(input_image, ret_bsize=True)

        weathered = []
        weathered_old = np.copy(input_image)
        while len(weathered) < 3:
            for i in range(5):
                weathered_cur = weathering(weathered_old, age_map=age_map, intact_texture_img=input_intact, bsize=block_size, use_median_age=False, do_texture_transfer=True, size=None)
                weathered_old = weathered_cur.copy()
            weathered.append(weathered_cur)

        deweathered = []
        for t in [0.5, 0.9, 0.95]:
            deweathered_t = deweather(input_image, t, details, gradient, input_intact, block_size, age_map)
            deweathered.append(deweathered_t)

        plt.figure(figsize=(20, 10))
        plt.subplot(171).imshow(deweathered[0])
        plt.title("<-- Deweathering --")
        plt.subplot(172).imshow(deweathered[1])
        plt.subplot(173).imshow(deweathered[2])
        plt.subplot(174).imshow(input_image)
        plt.title(img.split('/')[-1])
        plt.subplot(175).imshow(weathered[0])
        plt.subplot(176).imshow(weathered[1])
        plt.subplot(177).imshow(weathered[2])
        plt.title("-- Weathering -->")
        plt.show()
        
        if write_output:
            import os
            from pathlib import Path
            
            cwd = Path(os.pardir)
            output_dir = os.path.join(cwd, 'images', 'outputs', img.split('/')[-1].split('.')[0])
            
            try:
                os.mkdir(output_dir)
            except:
                pass
        
            indx = 0
            for w_img in weathered:
                indx += 1
                outPath = os.path.join(output_dir, "/weathered" + str(indx) + ".png")
                cv2.imwrite(outPath, cv2.cvtColor(w_img, cv2.COLOR_RGB2BGR))

            indx = 0
            for d_img in deweathered:
                indx += 1
                outPath = os.path.join(output_dir, "/deweathered" + str(indx) + ".png")
                cv2.imwrite(outPath, cv2.cvtColor(d_img, cv2.COLOR_RGB2BGR))   

In [ ]:
# Generating Outputs

img_paths = ['../images/' + x for x in ['texture0.png', 'texture1.png', 'texture2.png', 'texture3.png', 'texture4.png', 'texture5.png', 'texture6.png', 'texture7.png']]
run(img_paths, write_output=True)
